In [1]:
import pandas as pd
import numpy as np
import wandb
import os
os.environ["WANDB_DISABLED"] = "true"

import random
import os
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from datasets import Dataset
import time

In [2]:
df = pd.read_pickle('sports_news_KoBART.pkl')

In [3]:
print(df.shape); df.head()

(76, 4)


,TITLE,CONTENT,PUBLISH_DT,KoBART
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹를 주목했다. 수비수가 볼을 터치하지 않았다고 봤다. 멜레의 오른쪽 다리...,2021-07-15,"로세티 위원장이 심판을 대표해 의견을 냈지만, 오심이라고 생각하는 이들이 받아들일지..."
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난달...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 ‘스포츠 바...,2021-07-15,시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 20...,2021-07-15,"독일 분데스리가 마인츠, 호펜하임을 비롯해 잉글랜드 프리미어리그 크리스탈 팰리스가 ..."
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다.바...,2021-07-15,FC 바르셀로나가 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨이 킷을 공...


In [4]:
df = df[['KoBART', 'CONTENT', 'TITLE']]

In [5]:
train_df, val_df = train_test_split(df, random_state=42,test_size=.2)

In [6]:
df_train = pd.DataFrame({"title": train_df["TITLE"],'text':train_df["CONTENT"], 'sum':train_df["KoBART"]})
dataset_train = Dataset.from_pandas(df_train)

In [7]:
df_val = pd.DataFrame({"title": train_df["TITLE"],'text':train_df["CONTENT"], 'sum':train_df["KoBART"]})
dataset_val = Dataset.from_pandas(df_val)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [9]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["sum"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [10]:
tokenized_train_df = dataset_train.map(preprocess_function, batched=True)
tokenized_val_df = dataset_val.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_df,
    eval_dataset=tokenized_val_df,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: sum, __index_level_0__, text, title.
***** Running training *****
  Num examples = 60
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12


Epoch,Training Loss,Validation Loss
1,No log,0.410561
2,No log,0.384866
3,No log,0.376557


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: sum, __index_level_0__, text, title.
***** Running Evaluation *****
  Num examples = 60
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: sum, __index_level_0__, text, title.
***** Running Evaluation *****
  Num examples = 60
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: sum, __index_level_0__, text, title.
***** Running Evaluation *****
  Num examples = 60
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12, training_loss=0.5041841665903727, metrics={'train_runtime': 4.2219, 'train_samples_per_second': 42.635, 'train_steps_per_second': 2.842, 'total_flos': 37925310431232.0, 'train_loss': 0.5041841665903727, 'epoch': 3.0})

In [14]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: sum, __index_level_0__, text, title.
***** Running Evaluation *****
  Num examples = 60
  Batch size = 16


{'eval_loss': 0.3765568733215332,
 'eval_runtime': 0.4324,
 'eval_samples_per_second': 138.744,
 'eval_steps_per_second': 9.25,
 'epoch': 3.0}